In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import string
import nltk
import pickle

In [32]:
def remove_punctuatios(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text


In [33]:
with open('../static/model/model.pickle', 'rb') as file:
    model = pickle.load(file)

In [34]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [35]:
vocab = pd.read_csv('../static/model/vocab.txt',header = None)
tokens = vocab[0].tolist()

In [36]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()


In [37]:
def preprocessing(text):
    data = pd.DataFrame([text], columns =['tweet'])
    data['tweet'] = data['tweet'].apply(lambda x:" ".join(x.lower() for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x:" ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags =re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuatios)
    data['tweet'] = data['tweet'].str.replace('\d+', '',regex = True)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data['tweet']

In [38]:
def vectorizer(ds,vocab):
    vec_list = []
    
    for sent in ds:
        sent_list = np.zeros(len(vocab))
        
        for i in range(len(vocab)):
            if vocab[i] in sent.split():
                sent_list[i] = 1
        vec_list.append(sent_list)
    vec_list_new = np.asarray(vec_list,dtype = np.float32)
    return vec_list_new

In [46]:
def get_pred(vec_text):
    prediction = model.predict(vec_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [43]:
txt = 'Awesome I like it'

In [47]:
new_text = preprocessing(txt)
vec_text = vectorizer(new_text,tokens)
prediction = get_pred(vec_text)

In [48]:
prediction

'positive'